In [1]:
## Implementing JPEG

In [1]:
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
import math 
from matplotlib.colors import Normalize
import matplotlib.cm as cm
np.set_printoptions(precision=2,suppress=True)



In [10]:
### Find the size of the input imgae
print(f"The size of the input image is {os.path.getsize('cameraman.tif')} bytes")

The size of the input image is 65240 bytes


In [3]:
def DCTcoefficient(gray_img,B=8):
    h,w = gray_img.shape
    blocksV=h//B
    blocksH=w//B
    vis0 = np.zeros((h,w), np.float32)
    Trans = np.zeros((h,w), np.float32)
    vis0[:h, :w] = gray_img
    for row in range(blocksV):
            for col in range(blocksH):
                    currentblock = cv2.dct(vis0[row*B:(row+1)*B,col*B:(col+1)*B])
                    Trans[row*B:(row+1)*B,col*B:(col+1)*B]=currentblock
    cv2.imwrite('Transformed.jpg', Trans)
    return Trans
# cv2.cv.SaveImage('Transformed.jpg', cv2.cv.fromarray(Trans))

In [4]:
def Qmatrix(a,b,c):
    #     a,b,c= 10,40,20
    Q = np.array([[c,a,b,b,b,b,b,b],
     [a,a,b,b,b,b,b,b],
    [b,b,b,b,b,b,b,b],
    [b,b,b,b,b,b,b,b],
    [b,b,b,b,b,b,b,b],
    [b,b,b,b,b,b,b,b],
    [b,b,b,b,b,b,b,b],
    [b,b,b,b,b,b,b,b]])
    return Q

In [5]:
## Quantization
def quantize(A,Q):
    h,w = A.shape
    y = np.zeros((h,w), np.float32)
    for i in range(Q.shape[0]):
        for j in range(Q.shape[1]):
            y[i,j] = np.floor((A[i,j]/Q[i,j])  + 0.5)
    return y
    

In [6]:
# Lossless source coding
# A matrix has all the quantized dct coefficients

def encode_single(x):
    x = np.int16(x)
    if(x == 0):
        return '0'
    l = int(np.log2(np.abs(x)))
    if(x < 0):
        b = np.binary_repr((abs(x) ^ 2**(l+1)-1),width = l+1)
    else:
        b = np.binary_repr(x)
    return (l+1)*'1' + '0' + b

# print(encode_single(-7))

In [7]:

def bitstream(A,B=8):
    h,w = A.shape
    blocksV=h//B
    blocksH=w//B
    output_stream = ""
    for row in range(blocksV):
            for col in range(blocksH):
                z = A[row*B:(row+1)*B,col*B:(col+1)*B].flatten()
                for i in range(len(z)):
                    output_stream = output_stream+encode_single(z[i])
    with open('bitstream.txt', 'w') as f:
        f.write(output_stream)
    return output_stream
      

In [8]:
def decode(os):
    m = len(os)
    count = 1
    onecount = 0
    x= 1
    D = np.zeros(m)
    while x< m+1:
        a = os[x-1]
        if a ==1:
            onecount+=1
        elif a==0 and onecount ==0:
            D[count] = 0
        elif a==0 and onecount>0:
            l = 2**onecount
#             s= os(x:x+1)#(x+1:x+onecount)
            b = binaryToDecimal(s)
            if b<0.5:
                D[count] = b-l+1
            else:
                D[count] = b
            count+=1
            x+=onecount
            onecount = 0
        x+=1
    return D
        
                

In [9]:
def requantize(A,Q,B=8):
    h,w = A.shape
    blocksV=h//B
    blocksH=w//B
    R = np.zeros((h,w), np.float32)
    for row in range(blocksV):
        for col in range(blocksH):
                currentblock = A[row*B:(row+1)*B,col*B:(col+1)*B]
                R[row*B:(row+1)*B,col*B:(col+1)*B]=currentblock*Q
    return R
    

#     m,n = A.shape
#     nstride = m//8
#     inc = 8-1
#     icol = 1
#     for x in range(nstride):
#         irow = 0
#         for y in range(nstride):
#             A1 = A[irow:irow+inc,icol:icol+inc]
#             A[irow:irow+inc,icol:icol+inc] = A1*Q
    
    
    


In [10]:
def iDCTCoefficient(A,B=8):
    h,w = A.shape
    blocksV=h//B
    blocksH=w//B
    vis0 = np.zeros((h,w), np.float32)
    Trans = np.zeros((h,w), np.float32)
    vis0[:h, :w] = gray_img
    for row in range(blocksV):
            for col in range(blocksH):
                    currentblock = cv2.idct(vis0[row*B:(row+1)*B,col*B:(col+1)*B])
                    Trans[row*B:(row+1)*B,col*B:(col+1)*B]=currentblock
    return Trans


In [11]:
def rearrange(decode):
    B = np.reshape(decode, (-1, 2))
    return B
    

In [14]:
B=8 #blocksize
img = cv2.imread('cameraman.tif')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
h,w = gray_img.shape
dctI = DCTcoefficient(gray_img,B=8)
a,b,c= 10,40,20
Q = Qmatrix(a,b,c)
print(f"shape of Q-matrix{Q.shape}")

qdctI = quantize(dctI,Q)
print(f"shape of qdctI matrix {qdctI.shape}")

ostream = bitstream(qdctI ,B=8)
print(f"shape of ostream matrix {len(ostream)}")

decodes= decode(ostream)
print(f"shape of decodes matrix {decodes.shape}")

decodeI = rearrange(decodes)
print(f"shape of decodeI matrix {decodeI.shape}")

rdctI = requantize(decodeI,Q, B=8)
print(f"shape of rdctI matrix{rdctI.shape}")
IdctI = iDCTCoefficient(rdctI)


    


shape of Q-matrix(8, 8)
shape of qdctI matrix (256, 256)
shape of ostream matrix 65548
shape of decodes matrix (65548,)
shape of decodeI matrix (32774, 2)
shape of rdctI matrix(32774, 2)


ValueError: could not broadcast input array from shape (256,256) into shape (32774,2)

In [ ]:
# print(decode.shape)
# 256*256

In [ ]:
## Reconstruction of x
# xhat = np.zeros((h,w), np.float32)
# for i in range(Q.shape[0]):
#     for j in range(Q.shape[1]):
#         xhat[i,j] = y[i,j]*Q[i,j]


In [ ]:
# row, col = gray_img.shape
# newI = np.zeros(shape=gray_img.shape, dtype=np.uint8)
# for k1 in range(row):
#     for k2 in range(col):
#         for n1 in range(row):
#             for n2 in range(col):
#                 newI[k1][k2]+=gray_img[n1][n2]*math.cos(math.pi*k1*(n1+0.5)/row)*math.cos(math.pi*k2*(n1+0.5)/col)